# Introduction to Computation Graphs
- At the core of every TF program is the **computation graph**.
- A computation graph is a specific type of directed graph that is used for:
    - defining, unsurprisingly, computational structure
    
![](https://i.imgur.com/v9cLnlB.jpg)

The above example can be looked at as a simple equation:
$$f(1, 2) = 1 + 2 = 3$$

The fundamental building blocks of TF graphs are:
- **Nodes:**
    - Typically drawn as circles, ovals, or boxes
    - Represent some sort of computation or action being done on or with data in the graph's context.
    
- **Edges:**
    - actual values that get passed to and from Operations
    - typically drawn as arrows

# Defining Computation Graphs in TensorFlow
TensorFlow workflow pattern follows two steps:
- Define the computation graph
- Run the graph (with data)

# Building a TensorFlow Graph
![](https://i.imgur.com/ECJX8Iz.jpg)

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
%matplotlib inline 
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

In [2]:
a = tf.constant(5, name="input_a")
b = tf.constant(3, name="input_b")
c = tf.multiply(a, b, name="mul_c")
d = tf.add(a, b, name="add_d")
e = tf.add(c, d, name="add_e")

In [3]:
sess = tf.Session()
sess.run(e)

23

In [4]:
sess.run(c)

15

In [5]:
output = sess.run(e)

In [6]:
# TensorBoard config
writer = tf.summary.FileWriter('./my_graph', sess.graph)

In [7]:
writer.close()
sess.close()

# Thinking with Tensors
- 1-D Tensor --> a vector
- 2-D Tensor --> a matrix

![](https://i.imgur.com/gzhEMCY.jpg)

This graph has several advantages over the previous example:
- The client only has to send input to a single node, which simplifies using the graph.
- The nodes that directly depend on the input now only have to keep track of one dependency instead of two.
- We now have the option of making the graph take in vectors of any length, if we'd like. This would make the graph more flexible.
- In TF, all the data passed from node to node are **Tensor** objects.
- Tensors are just a superset of matrices!

In [8]:
a = tf.constant([5, 3], name="input_a")
b = tf.reduce_prod(a, name="prod_b")
c = tf.reduce_sum(a, name="sum_c")
d = tf.add(b, c, name="add_d")

In [9]:
sess = tf.Session()

In [10]:
sess.run(d)

23

# Python Native Types
- TF can take in Python numbers, booleans, strings, or lists of any of the above.
- Single values --> 0-D Tensor(or scalar)
- list of values --> 1-D Tensor(vector)
- list of lists of values -> 2-D Tensor(matrix)

# NumPy arrays
- TF is tightly integrated with **NumPy**
- TF data types are based on those from NumPy

In [11]:
np.int32 == tf.int32

True

- Any NumPy array can be passed into any TF Op.

# TensorFlow operations
- Are nodes that perform computations on or with Tensor objects
- After computation, they return zero or more tensors.

## Overloaded operators
- TF also overloads common mathematical operators to make multiplication, addition, subtraction, and other common operations more concise.



# TensorFlow graphs
 Creating a Graph is simple-its constructor doesn't take any variables:

In [12]:
# create a new graph
g = tf.Graph()

Once we have our Graph initialized, we can add Operations to it by using the ```Graph.as_default()``` method to access its context manager.

In [13]:
with g.as_default():
    # create Operations as usual; they will be adding to graph 'g'
    a = tf.multiply(2, 3)

- TF automatically creates a ```Graph``` when the library is loaded and assigns it to be the default. Thus, any Operations, tensors, etc. defined outside of a ```Graph.as_default()``` context manager will automatically be placed in the default graph:

In [14]:
# Placed in the default graph
in_default_graph = tf.add(1, 2)

In [15]:
# Placed in graph 'g'
with g.as_default():
    in_graph_g = tf.multiply(2, 3)

In [16]:
# We are no longer in the with block, so this is placed in the default graph
also_in_default_graph = tf.subtract(5, 1)

In [17]:
# To get a handle to the default graph, use the ```tf.get_default_graph()``` function:
default_graph = tf.get_default_graph()

# TensorFlow Sessions
- Are responsible for graph execution.
- The constructor takes in three optional parameters:
    - ```target``` specifies the execution engine to use
    - ```graph``` specifies the ```Graph``` object that will be launched in the ```session```. The default value is ```None```.
    - ```config``` allows users to specify options to configure the session, such as limiting the number of CPUs or GPUs to use, setting optimization parameters for graphs, and logging options.

In [18]:
a = tf.add(2, 5)
b = tf.multiply(a, 3)

# Start up a session using the default graph
sess = tf.Session()

Once a **Session** is opened, you can use its primary method, ```run()```, to calculate the value of a desired ```Tensor``` output

In [19]:
sess.run(b)

21

```Session.run()``` takes in one required parameter, **```fetches```**, as well as three optional parameters:
- ```feed_dict```
- ```options```
- ```run_metadata```

## 1. Fetches
- ```fetches``` accepts any graph element (either an operation or Tensor object), which specifies what the user would like to execute.
- If the requested object is a **Tensor**, then the output of ```run()``` will be a NumPy array.
- If the object is an ```Operation```, then the output will be ```None```.

In [20]:
sess.run(b) # we set fetches to the tensor b

21

In [21]:
sess.run([a, b]) # pass in a list of graph elements

[7, 21]

- When ```fetches``` is a list, the output of ```run()``` will be a list with values corresponding to the output of the requested elements.
- In addition using ```fetches``` to get ```Tensor``` outputs, we can give ```fetches``` a direct handle to an **operation** whcih is a useful side-effect when run.
    - Example: ```tf.global_variables_initializer()```

In [22]:
# Performs the computations needed to initialize Variables, but returns ```None```
sess.run(tf.global_variables_initializer())

## 2. Feed Dictionary
- The parameter ```feed_dict``` is used to override ```Tensor``` values in the graph, and it expects a Python dictionary object as input.
- The keys in the dictionary are handles to ```Tensor``` objects that should be overridden, while the values can be numbers, strings, lists, or NumPy arrays.
- The values must of the same type.

In [23]:
# create Operations, Tensors, etc (using teh default graph)
a = tf.add(2, 5)
b = tf.multiply(a, 3)

In [24]:
# Start up a Session using the default graph
sess = tf.Session()

In [25]:
# Define a dictionary that says to replace the value of a with 15
replace_dict = {a:15}


In [26]:
# Run the session, passing in replace_dict as the value to feed_dict
sess.run(b, feed_dict=replace_dict)

45

In [27]:
sess.close()

# Adding Inputs with Placeholder nodes
- Placeholders act as if they are Tensor objects, but they do not have their values specified when created.
- Instead, they hold the place for a Tensor that will be fed at runtime, in effect becoming an **input** node.

In [28]:
# Creates a placeholder vector of length 2 with data type int32
a = tf.placeholder(tf.int32, shape=[2], name="my_input")

In [29]:
# Use the placeholder as if it were any other Tensor object
b = tf.reduce_prod(a, name="prod_b")
c = tf.reduce_sum(a, name="sum_c")

In [30]:
# Finish off the graph
d = tf.add(b, c, name="add_d")

```tf.placeholder``` takes in a required parameter ```dtype```, as well as the optional parameter ```shape```:

- ```dtype``` specifies the data type of values that will be passed into the placeholder. This is required, as it is needed to ensure that there will be no type mismatch errors.
- ```shape``` specifies what shape the fed Tensor will be. The default value of ```shape``` is ```None```, which means a Tensor of any shape will be accepted.

In [31]:
# Open a TF Session
sess = tf.Session()

In [32]:
# Create a dictionary to pass into ```feed_dict```
# Key: `a`, the handle to the placeholder's output Tensor
# Value: A vector with value [5,3] and int32 data type

input_dict = {a: np.array([5, 3], dtype=np.int32)}

In [33]:
# Fetch the value of `d`, feeding the values of `input_vector` into `a`
sess.run(d, feed_dict=input_dict)

23

- You must include a key-value pair in ```feed_dict``` for each placeholder that is a dependency of the fetched output.

# Variables
## Creating variables

- ```Tensor``` and ```Operation``` objects are immutable, but machine learning tasks, by their nature, need a mechanism to save changing values over time.
- This is accomplished in TF with ```variable``` objects, which contain mutable tensor values that persist across multiple calls to ```Session.run()```

In [34]:
# Pass in a starting value of three for the variable
my_var = tf.Variable(3, name="my_variable")

In [35]:
add = tf.add(5, my_var)
mul = tf.multiply(8, my_var)

- The initial value of variables will often be large tensors of zeros, ones, or random values.
- Each TF Ops takes in a **shape** parameter which specifies the dimension of the desired **Tensor**:

In [36]:
# 2x2 matrix of zeros
zeros = tf.zeros([2, 2])

In [37]:
# vector of length 6 of ones
ones = tf.ones([6])

In [38]:
# 3x3x3 Tensor of random uniform values between 0 and 10
uniform = tf.random_uniform([3, 3, 3], minval=0, maxval=10)

In [39]:
# 3x3x3 Tensor of normally distributed numbers; mean 0 and standard deviation 2
normal = tf.random_normal([3, 3, 3], mean=0.0, stddev=2.0)

- Instead of using ```tf.random_normal()```, we often use ```tf.truncated_normal()``` instead, as it doesn't create any values more than two standard deviations away from its mean.
- This prevents the possibility of having one or two numbers be significantly different than the other values in the tensor:

In [40]:
# No values below 3.0 or above 7.0 will be returned in this Tensor
trunc = tf.truncated_normal([2, 2], mean=5.0, stddev=1.0)

## Variable Initialization
- ```variable``` objects live in the ```Graph``` like most other TF objects, but their state is actually managed by a ```Session```. Because of this, variables have an extra step involved in order to use them- you must initialize the ```variable```. 
- This is typically done by passing in the ```tf.global_variables_initializer()``` operation to ```Session.run()```:

In [41]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

- If you'd only like to initialize a subset of Variables defined in the graph, you can use ```tf.variables_initializer()```, which takes in a list of Variables to be initialized:

In [42]:
var1 = tf.Variable(0, name="initialize_me")
var2 = tf.Variable(1, name="no_initialization")
init = tf.variables_initializer([var1], name="init_var1")
sess = tf.Session()
sess.run(init)
sess.close()

## Changing Variables
- In order to change the value of the ```Variable```, you can use the ```Variable.assign()``` method, which gives the ```Variable``` the new value to be.
- Note that ```Variable.assign()``` is an Operation, and must be run in a ```Session``` to take effect:

In [43]:
# Create variable with starting value of 1
my_var = tf.Variable(1)

# Create an operation that multiplies the variable by 2 each time it is run
my_var_times_two = my_var.assign(my_var * 2)

In [44]:
# Initialization operation
init = tf.global_variables_initializer()

In [45]:
# Start a session
sess = tf.Session()

# Initialize variable
sess.run(init)

In [46]:
# Multiply variable by two and return it
sess.run(my_var_times_two)

2

In [47]:
# Multiply again
sess.run(my_var_times_two)

4

In [48]:
# Multiply again
sess.run(my_var_times_two)

8

- For simple incrementing and decrementing of Variables, TF includes the ```Variable.assign_add()``` and ```Variable.assign_sub()```.

In [49]:
# Increment by 1
sess.run(my_var.assign_add(1))

9

In [50]:
# Decrement by 2
sess.run(my_var.assign_sub(2))

7

In [51]:
sess.close()

- Because Sessions maintain ```variable``` values separately, each ```Session``` can have its own current value for a ```Variable``` defined in a graph:

In [52]:
# Create Ops
my_var = tf.Variable(0)
init = tf.global_variables_initializer()

In [53]:
# Start Sessions
sess1 = tf.Session()
sess2 = tf.Session()

In [54]:
# Initialize variable in sess1, and increment value of my_var in that Session
sess1.run(init)
sess1.run(my_var.assign_add(5))

5

In [55]:
# Do the same with sess2, but use a different increment value
sess2.run(init)
sess2.run(my_var.assign_add(2))

2

In [56]:
# Can increment the Variable values in each Session independently
sess1.run(my_var.assign_add(5))

10

In [57]:
sess2.run(my_var.assign_add(2))

4

- If you'd like to reset your Variables to their  starting value, simply call ```tf.global_variables_initializer()```

In [58]:
# Create Ops
my_var = tf.Variable(0)
init = tf.global_variables_initializer()

In [59]:
# Start Session
sess = tf.Session()


In [60]:
# Initialize Variables
sess.run(init)

In [61]:
# Change the variable
sess.run(my_var.assign(10))

10

In [62]:
# Reset the Variable to 0, its initial value
sess.run(init)

In [63]:
sess.close()

# Organizing your graph with name scopes
- Real world models can contain dozens or hundreds of nodes, as well as millions of parameters.
- In order to manage this level of complexity, TF currently offers a mechanism to help organize your graphs: **name scopes**.
- Name scopes are incredibly simple to use and provide great value when visualizing your graph with TensorBoard.
- Name scopes allow you to group Operations into larger, named blocks. Then, when you launch your graph with TensorBoard, each name scope will encapsulate its own Ops, making the visualization much more digestible.

In [69]:
with tf.name_scope("Scope_A"):
    a = tf.add(1, 2, name="A_add")
    b = tf.multiply(a, 3, name="A_mul")

In [70]:
with tf.name_scope("Scope_B"):
    c = tf.add(4, 5, name="B_add")
    d = tf.multiply(c, 6, name="B_mul")

In [71]:
e = tf.add(b, d, name="output")

In [72]:
writer = tf.summary.FileWriter('./name_scope_1', 
                               graph=tf.get_default_graph())
writer.close()

- Because the ```tf.summary.FileWriter()``` exports the graph immediately, we can simply start up TensorBoard after running the above code.

- Navigate to where the ```name_scope_1``` directory is created and start up TensorBoard:
```bash
$ tensorboard --logdir='./name_scope_1'
```

![](https://i.imgur.com/F10aIbC.jpg)